In [16]:
from station_dataframe_generator import StationDataframeGenerator
import numpy as np
import pandas as pd

res = StationDataframeGenerator.generate_station_dataframe(813, range(2019, 2022))
pm10_data_station_814 = StationDataframeGenerator.generate_station_dataframe(814, range(2019, 2022))


In [5]:
np.where(np.isnan(pm10_data_station_814["pm10"].to_numpy()) == True)[0]


array([ 3982,  6816,  6817,  6818,  6819,  6820,  6821,  6822,  6823,
        6824,  6825,  6826,  6827,  6828,  6829,  6830,  6831,  6832,
        6833,  6834,  6835,  6836,  6837,  6838,  6839,  6840,  6841,
        6842,  6843,  6844,  6845,  6846,  6847,  6848,  6849,  6850,
        6851,  6852,  6853,  6854,  6855,  6856,  6857,  6858,  6859,
        6860,  6861,  6862,  6863,  6864,  6865,  6866,  6867,  6868,
        6869,  6870,  6871,  6872,  6873,  6874,  6875,  6876,  6877,
        6878,  6879,  6880,  6881,  6882,  6883,  6884,  6885,  6886,
        6887,  6888,  6889,  6890,  6891,  6892,  6893,  6894,  6895,
        6896,  6897,  6898,  6899,  6900,  6901,  6902,  6903,  6904,
        6905,  6906,  6907,  6908,  6909,  6910,  6911,  6912,  6913,
        6914,  6915,  6916,  6917,  6918,  6919,  6920,  6921,  6922,
        6923,  6924,  6925,  6926,  6927,  6928,  6929,  6930,  6931,
        6932,  6933,  6934,  6935,  6936,  6937,  6938,  6939,  6940,
        6941,  6942,

## Umgang mit fehlenden PM10 Werten

- Interpolation einzelner fehlender Werte (Nan) mit Sliding Window
- Restliche NaN mit df.dropna() entfernen


In [6]:
pm10_data_station_814_interpolated = pm10_data_station_814.interpolate(
    method="time", limit=5, limit_area="inside", inplace=False
)
# pm10_data_station_814_interpolated.reset_index(inplace=True)


In [7]:
# Beispiel DataFrame
df = pm10_data_station_814

# Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten interpolieren
max_gap_size = 5

# Finde die Indizes der NaN-Werte
nan_indices = df[df["pm10"].isnull()].index
nan_indices


DatetimeIndex(['2019-06-15 23:00:00', '2019-10-12 01:00:00',
               '2019-10-12 02:00:00', '2019-10-12 03:00:00',
               '2019-10-12 04:00:00', '2019-10-12 05:00:00',
               '2019-10-12 06:00:00', '2019-10-12 07:00:00',
               '2019-10-12 08:00:00', '2019-10-12 09:00:00',
               ...
               '2021-08-09 01:00:00', '2021-08-09 02:00:00',
               '2021-08-09 03:00:00', '2021-08-09 04:00:00',
               '2021-08-09 05:00:00', '2021-08-09 06:00:00',
               '2021-08-09 07:00:00', '2021-08-09 08:00:00',
               '2021-08-09 09:00:00', '2021-08-09 10:00:00'],
              dtype='datetime64[ns]', name='time', length=647, freq=None)

In [8]:
# Beispiel DataFrame
df = pm10_data_station_814.copy()

# Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten interpolieren
max_gap_size = pd.Timedelta(hours=5)

# Finde die Indizes der NaN-Werte
nan_indices = df[df["pm10"].isnull()].index

# Finde die Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten
gap_start_indices = []
current_gap_start = None
last_index = None

for index in nan_indices:
    if current_gap_start is None:
        current_gap_start = index
        last_index = index
    elif index - last_index > pd.Timedelta(hours=1):
        if last_index - current_gap_start < max_gap_size:
            gap_start_indices.append([current_gap_start, last_index])
            current_gap_start = index
        else:
            current_gap_start = index

    last_index = index

gap_start_indices


[[Timestamp('2019-06-15 23:00:00'), Timestamp('2019-06-15 23:00:00')],
 [Timestamp('2019-11-05 05:00:00'), Timestamp('2019-11-05 05:00:00')],
 [Timestamp('2019-11-16 20:00:00'), Timestamp('2019-11-16 20:00:00')],
 [Timestamp('2020-04-22 14:00:00'), Timestamp('2020-04-22 15:00:00')],
 [Timestamp('2020-11-06 10:00:00'), Timestamp('2020-11-06 11:00:00')],
 [Timestamp('2021-01-01 16:00:00'), Timestamp('2021-01-01 17:00:00')],
 [Timestamp('2021-06-07 11:00:00'), Timestamp('2021-06-07 12:00:00')],
 [Timestamp('2021-06-17 22:00:00'), Timestamp('2021-06-17 22:00:00')]]

In [9]:
# Interpoliere die Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten
for start_index, end_index in gap_start_indices:
    length = end_index - start_index
    if length < pd.Timedelta(hours=2):
        length = pd.Timedelta(hours=2)
    real_start = start_index - length
    real_end = end_index + length
    df.loc[real_start:real_end, "pm10"] = df.loc[real_start:real_end, "pm10"].interpolate(method="time")

df


,temperature_2m,relativehumidity_2m,winddirection_10m,windspeed_10m,precipitation,pm10
time,,,,,,
2019-01-01 01:00:00,0.9,83,265,13.7,0.0,84.9443
2019-01-01 02:00:00,0.6,82,257,14.8,0.0,59.5450
2019-01-01 03:00:00,0.3,83,251,15.6,0.0,55.2941
2019-01-01 04:00:00,0.3,85,248,15.2,0.0,39.3113
2019-01-01 05:00:00,0.0,87,237,15.9,0.0,35.3945
...,...,...,...,...,...,...
2021-12-31 20:00:00,9.3,91,260,24.1,0.0,18.2603
2021-12-31 21:00:00,9.6,89,260,23.7,0.0,17.5962
2021-12-31 22:00:00,9.5,87,261,24.1,0.0,17.5859


In [10]:
df_dropped = df.dropna()
df_dropped


,temperature_2m,relativehumidity_2m,winddirection_10m,windspeed_10m,precipitation,pm10
time,,,,,,
2019-01-01 01:00:00,0.9,83,265,13.7,0.0,84.9443
2019-01-01 02:00:00,0.6,82,257,14.8,0.0,59.5450
2019-01-01 03:00:00,0.3,83,251,15.6,0.0,55.2941
2019-01-01 04:00:00,0.3,85,248,15.2,0.0,39.3113
2019-01-01 05:00:00,0.0,87,237,15.9,0.0,35.3945
...,...,...,...,...,...,...
2021-12-31 20:00:00,9.3,91,260,24.1,0.0,18.2603
2021-12-31 21:00:00,9.6,89,260,23.7,0.0,17.5962
2021-12-31 22:00:00,9.5,87,261,24.1,0.0,17.5859


In [11]:
pm10_data_station_814_interpolated.iloc[6810:]


,temperature_2m,relativehumidity_2m,winddirection_10m,windspeed_10m,precipitation,pm10
time,,,,,,
2019-10-11 19:00:00,12.8,62,228,17.3,0.0,13.7238
2019-10-11 20:00:00,12.1,64,224,16.0,0.0,12.7065
2019-10-11 21:00:00,11.4,66,220,14.1,0.0,12.6222
2019-10-11 22:00:00,10.8,68,214,13.0,0.0,12.1732
2019-10-11 23:00:00,10.3,71,207,12.6,0.0,11.9590
...,...,...,...,...,...,...
2021-12-31 20:00:00,9.3,91,260,24.1,0.0,18.2603
2021-12-31 21:00:00,9.6,89,260,23.7,0.0,17.5962
2021-12-31 22:00:00,9.5,87,261,24.1,0.0,17.5859


In [12]:
pm10_data_station_814.dropna()


,temperature_2m,relativehumidity_2m,winddirection_10m,windspeed_10m,precipitation,pm10
time,,,,,,
2019-01-01 01:00:00,0.9,83,265,13.7,0.0,84.9443
2019-01-01 02:00:00,0.6,82,257,14.8,0.0,59.5450
2019-01-01 03:00:00,0.3,83,251,15.6,0.0,55.2941
2019-01-01 04:00:00,0.3,85,248,15.2,0.0,39.3113
2019-01-01 05:00:00,0.0,87,237,15.9,0.0,35.3945
...,...,...,...,...,...,...
2021-12-31 20:00:00,9.3,91,260,24.1,0.0,18.2603
2021-12-31 21:00:00,9.6,89,260,23.7,0.0,17.5962
2021-12-31 22:00:00,9.5,87,261,24.1,0.0,17.5859


In [13]:
res["pm10"].isnull().sum()


86

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df2019 = pd.read_json("../../../data/pollution/raw/2019/pm10/meta_data.json")
df2020 = pd.read_json("../../../data/pollution/raw/2020/pm10/meta_data.json")
df2021 = pd.read_json("../../../data/pollution/raw/2021/pm10/meta_data.json")

fig_model = make_subplots(column_titles=["Min Max values"])

fig_model.add_trace(go.Bar(x=df2019.index, y=df2019["max"], name="Max 2019"))
fig_model.add_trace(go.Bar(x=df2020.index, y=df2020["max"], name="Max 2020"))
fig_model.add_trace(go.Bar(x=df2021.index, y=df2021["max"], name="Max 2021"))

fig_model.add_trace(go.Bar(x=df2019.index, y=df2019["min"], name="Min 2019"))
fig_model.add_trace(go.Bar(x=df2020.index, y=df2020["min"], name="Min 2020"))
fig_model.add_trace(go.Bar(x=df2021.index, y=df2021["min"], name="Min 2021"))


In [15]:
# Not correct

df2019 = pd.read_json("../../../data/pollution/raw/2019/pm25/meta_data.json")
df2020 = pd.read_json("../../../data/pollution/raw/2020/pm25/meta_data.json")
df2021 = pd.read_json("../../../data/pollution/raw/2021/pm25/meta_data.json")

fig_model = make_subplots(column_titles=["Min Max values (pm2.5)"])

fig_model.add_trace(go.Bar(x=df2019.index, y=df2019["max"], name="Max 2019"))
fig_model.add_trace(go.Bar(x=df2020.index, y=df2020["max"], name="Max 2020"))
fig_model.add_trace(go.Bar(x=df2021.index, y=df2021["max"], name="Max 2021"))

fig_model.add_trace(go.Bar(x=df2019.index, y=df2019["min"], name="Min 2019"))
fig_model.add_trace(go.Bar(x=df2020.index, y=df2020["min"], name="Min 2020"))
fig_model.add_trace(go.Bar(x=df2021.index, y=df2021["min"], name="Min 2021"))
